In [ ]:
!wget https://huggingface.co/danjacobellis/LCCL/resolve/main/vss_samples_rr_18x.pth

In [1]:
import io
import torch
import torchaudio
import torchvision
import numpy as np
from datasets import load_dataset
from walloc import walloc
from IPython.display import display, Audio, Image, update_display, HTML
from torchvision.transforms import ToPILImage
from fastprogress.fastprogress import master_bar, progress_bar
from ioae import IsotropicOobleckAutoencoder
from spauq.core.metrics import spauq_eval
from einops import rearrange
import cdpam
class Config: pass

In [2]:
device = "cuda"

cdpam_loss = cdpam.CDPAM()

checkpoint = torch.load("vss_samples_rr_18x.pth",map_location="cpu",weights_only=False)
config = checkpoint['config']
model = IsotropicOobleckAutoencoder(
    channels=config.channels,
    patch_size=config.patch_size,
    embed_dim=config.embed_dim,
    depth=config.depth
).to(device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()
sum(p.numel() for p in model.parameters())/1e6

/home/dgj335/.local/lib/python3.10/site-packages/cdpam/cdpam.py:45: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(modfolder,map_location="cpu")['state']
/

6.7132

In [3]:
valid_dataset = load_dataset("danjacobellis/musdb18hq_vss",split='validation')
if config.rate_reduction == 1:
    resample_down = torch.nn.Identity().to(device)
    resample_up = torch.nn.Identity().to(device)
else:
    resample_down = torchaudio.transforms.Resample(
        orig_freq=44100,
        new_freq=44100 // config.rate_reduction
    ).to(device)
    resample_up = torchaudio.transforms.Resample(
        orig_freq=44100 // config.rate_reduction,
        new_freq=44100
    ).to(device)

In [8]:
def eval_vss(sample):
    with torch.no_grad():
        sample = valid_dataset[200]
        xb, fs = torchaudio.load(sample['audio_mix']['bytes'])
        vb, fs = torchaudio.load(sample['audio_vocal']['bytes'])
        B = xb.shape[-1]//config.length_samples
        x = torch.zeros((B,2,config.length_samples))
        v = torch.zeros((B,2,config.length_samples))
        for i_frame in range(B):
            i_start = i_frame*config.length_samples
            i_end = (i_frame+1)*config.length_samples
            x[i_frame,:,:] = xb[:,i_start:i_end]
            v[i_frame,:,:] = vb[:,i_start:i_end]
        x = x.to(device)
        v = v.to(device)
        xc = resample_down(x)
        pred = model(xc)
        v_hat = resample_up(pred)
    
        v = rearrange(v, 'B C L -> C (B L)')/2
        v_hat = rearrange(v_hat, 'B C L -> C (B L)')
        # v_hat = (v.std()/v_hat.std())*v_hat
        
        PSNR = -10*np.log10(torch.nn.functional.mse_loss(v,v_hat).item())
        SDR = spauq_eval(v.cpu(),v_hat.cpu(),fs=fs)
        SSDR = SDR['SSR']
        SRDR = SDR['SRR']
        cdpam = cdpam_loss.forward(v,v_hat).mean().item()

        return {
            'PSNR': PSNR,
            'SSDR': SSDR,
            'SRDR': SRDR,
            'CDPAM': cdpam
        }

In [9]:
eval_results = valid_dataset.map(eval_vss)
eval_only = eval_results.remove_columns(['audio_mix', 'audio_vocal', 'path_mix'])

Map:   0%|          | 0/235 [00:00<?, ? examples/s]

In [10]:
# without normalization
print(np.median(eval_only['PSNR']))
print(np.median(eval_only['SSDR']))
print(np.median(eval_only['SRDR']))
print(np.median(-10*np.log10(eval_only['CDPAM'])))

28.917826074630252
7.34165313190401
-0.18208525194840308
40.49529437939107


In [11]:
# without normalization
print(np.mean(eval_only['PSNR']))
print(np.mean(eval_only['SSDR']))
print(np.mean(eval_only['SRDR']))
print(np.mean(-10*np.log10(eval_only['CDPAM'])))

28.917826074630245
7.34165313190401
-0.18208525194840314
40.49529437939107


In [12]:
eval_results.push_to_hub("danjacobellis/MUSDB_vss_samples_rr_18x",split='validation')

Uploading the dataset shards:   0%|          | 0/8 [00:00<?, ?it/s]

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

...

Map:   0%|          | 0/29 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Map:   0%|          | 0/29 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/danjacobellis/MUSDB_vss_samples_rr_18x/commit/a08bf0e688dbbd38e57954cffcf97bb758442ac0', commit_message='Upload dataset', commit_description='', oid='a08bf0e688dbbd38e57954cffcf97bb758442ac0', pr_url=None, pr_revision=None, pr_num=None)